In [89]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline

In [136]:
rootpath ='../../../data/third_party_preds/'
filename_list=['baidu-080815.csv','baidu-081515.csv','baidu-082215.csv','baidu-082815.csv',
               'baidu-091215.csv','baidu-091915.csv','baidu-092315.csv','baidu-092615.csv',
               'baidu-100315.csv','baidu-101715.csv','baidu-102415.csv','baidu-120415.csv']

In [137]:
#betting return
def betting_return(result_dummies, bookmaker_odds):
    odds = bookmaker_odds-1
    odds =pd.DataFrame(result_dummies.values*odds.values, columns=result_dummies.columns)
    odds =odds.replace([0.0],[-1])
    return odds

In [138]:
#method1
#1. Naive: We place a unit bet on the outcome predicted by the model for each game, that with maximum p.
def betting_with_max_p(filename, bookmaker=['b365H', 'b365D', 'b365A'], bankroll_unit = 1):
    all_data = pd.read_csv(rootpath+filename, encoding='cp1252')
    
    result_dummies = pd.get_dummies(all_data, columns=['result'])[['result_1', 'result_2', 'result_3']]    
    all_data['maximum p'] = all_data[['Home Win','Home Draw','Home Lost']].max(axis=1)
    
    all_data['bet_win'] = np.where((all_data['Home Win'] >= all_data['maximum p']) & (all_data['Home Draw']<all_data['maximum p']) & (all_data['Home Lost']<all_data['maximum p'])
                    , bankroll_unit, 0)
    all_data['bet_draw'] = np.where((all_data['Home Draw'] >= all_data['maximum p']) & (all_data['Home Win']<all_data['maximum p']) & (all_data['Home Lost']<all_data['maximum p'])
                    , bankroll_unit, 0)
    all_data['bet_lost'] = np.where((all_data['Home Lost'] >= all_data['maximum p']) & (all_data['Home Draw']<all_data['maximum p']) & (all_data['Home Win']<all_data['maximum p'])
                    , bankroll_unit, 0)
    odds = betting_return(result_dummies, all_data[bookmaker])
    all_data = pd.concat([all_data, odds],axis=1)
    
    all_data[['act_win','act_draw','act_lost']] =pd.DataFrame(odds.values*all_data[['bet_win','bet_draw','bet_lost']].values, columns=odds.columns, index=odds.index)   
    all_data['total'] = all_data['act_win']+all_data['act_draw']+all_data['act_lost']   

    actual_profit = (all_data['total']).sum()
    actual_betting_money = (all_data['bet_win']+all_data['bet_draw']+all_data['bet_lost']).sum()
    return all_data, actual_profit, actual_betting_money   

In [139]:
#method2
#2. Positive: We place a unit bet on all outcomes for which ∆i > 0（∆i = Pi − 1/Oddsi）
def betting_with_d_positive(filename,bookmaker=['b365H', 'b365D', 'b365A'], bankroll_unit = 1):
    all_data = pd.read_csv(rootpath+filename, encoding='cp1252')
    
    result_dummies = pd.get_dummies(all_data, columns=['result'])[['result_1', 'result_2', 'result_3']]    
    all_data['Di_win'] = all_data['Home Win']- 1/all_data[bookmaker[0]]
    all_data['Di_draw'] = all_data['Home Draw']- 1/all_data[bookmaker[1]]
    all_data['Di_lost'] = all_data['Home Lost']- 1/all_data[bookmaker[2]]
    
    all_data['bet_win'] = np.where(all_data['Di_win']>0, bankroll_unit, 0)
    all_data['bet_draw'] = np.where(all_data['Di_draw']>0, bankroll_unit, 0)
    all_data['bet_lost'] = np.where(all_data['Di_lost']>0, bankroll_unit, 0)
    
    odds = betting_return(result_dummies, all_data[bookmaker])
    all_data = pd.concat([all_data, odds],axis=1)
    
    all_data[['act_win','act_draw','act_lost']] =pd.DataFrame(odds.values*all_data[['bet_win','bet_draw','bet_lost']].values, columns=odds.columns, index=odds.index)   
    all_data['total'] = all_data['act_win']+all_data['act_draw']+all_data['act_lost']   

    actual_profit = (all_data['total']).sum()
    actual_betting_money = (all_data['bet_win']+all_data['bet_draw']+all_data['bet_lost']).sum()
    return all_data, actual_profit, actual_betting_money

In [144]:
#method3
"""3. Best: For each game, we place a unit bet on the outcome with maximum ∆i, if that
∆i > 0. This is similar to Positive, but places a bet on only one outcome per game.（∆i = Pi − 1/Oddsi）
"""
def betting_with_max_d_positive(filename,bookmaker=['b365H', 'b365D', 'b365A'], bankroll_unit = 1):
    all_data = pd.read_csv(rootpath+filename, encoding='cp1252')
    
    result_dummies = pd.get_dummies(all_data, columns=['result'])[['result_1', 'result_2', 'result_3']]    
    all_data['Di_win'] = all_data['Home Win']- 1/all_data[bookmaker[0]]
    all_data['Di_draw'] = all_data['Home Draw']- 1/all_data[bookmaker[1]]
    all_data['Di_lost'] = all_data['Home Lost']- 1/all_data[bookmaker[2]]
    
    all_data['maximum D'] = all_data[['Di_win','Di_draw','Di_lost']].max(axis=1)
    
    all_data['bet_win'] = np.where((all_data['Di_win'] >= all_data['maximum D'])&(all_data['Di_win']>0) & (all_data['Di_draw']<all_data['maximum D']) & (all_data['Di_lost']<all_data['maximum D'])
                    , bankroll_unit, 0)
    all_data['bet_draw'] = np.where((all_data['Di_draw'] >= all_data['maximum D'])&(all_data['Di_draw']>0) & (all_data['Di_win']<all_data['maximum D']) & (all_data['Di_lost']<all_data['maximum D'])
                    , bankroll_unit, 0)
    all_data['bet_lost'] = np.where((all_data['Di_lost'] >= all_data['maximum D'])&(all_data['Di_lost']>0) & (all_data['Di_draw']<all_data['maximum D']) & (all_data['Di_win']<all_data['maximum D'])
                    , bankroll_unit, 0)
    
    odds = betting_return(result_dummies, all_data[bookmaker])
    all_data = pd.concat([all_data, odds],axis=1)
    
    all_data[['act_win','act_draw','act_lost']] =pd.DataFrame(odds.values*all_data[['bet_win','bet_draw','bet_lost']].values, columns=odds.columns, index=odds.index)   
    all_data['total'] = all_data['act_win']+all_data['act_draw']+all_data['act_lost']   

    actual_profit = (all_data['total']).sum()
    actual_betting_money = (all_data['bet_win']+all_data['bet_draw']+all_data['bet_lost']).sum()
    return all_data, actual_profit, actual_betting_money

In [157]:
#method4
"""
4. K-Result: We place a bet on outcome i, bi = fiC/3k, if fi > 0 where k is the number of games 
(and 3k the number of outcomes) to be bet on.
C :总预算(bankroll)
fi:凯利指数，公式p - (1 - p) / (odd - 1) （p为概率，odd为赔率）
"""
def betting_with_k_result(filename,bookmaker=['b365H', 'b365D', 'b365A'], bankroll=1000):
    all_data = pd.read_csv(rootpath+filename, encoding='cp1252')
    
    result_dummies = pd.get_dummies(all_data, columns=['result'])[['result_1', 'result_2', 'result_3']]    
    
    #kelly_criterion
    all_data['k_win'] = all_data['Home Win']- (1- all_data['Home Win'])/(all_data[bookmaker[0]]-1)
    all_data['k_draw'] =all_data['Home Draw']- (1- all_data['Home Draw'])/(all_data[bookmaker[1]]-1)
    all_data['k_lost'] =all_data['Home Lost']- (1- all_data['Home Lost'])/(all_data[bookmaker[2]]-1)
    
    all_data['bet_win'] = np.where((all_data['k_win'] >0), (all_data['k_win']*bankroll)/(3*all_data.shape[0]), 0)
    all_data['bet_draw'] = np.where((all_data['k_draw'] >0), (all_data['k_draw']*bankroll)/(3*all_data.shape[0]), 0)
    all_data['bet_lost'] = np.where((all_data['k_lost'] >0), (all_data['k_lost']*bankroll)/(3*all_data.shape[0]), 0)
    #all_data['bet_win'] = np.where((all_data['k_win'] >0), all_data['k_win'], 0)
    #all_data['bet_draw'] = np.where((all_data['k_draw'] >0), all_data['k_draw'], 0)
    #all_data['bet_lost'] = np.where((all_data['k_lost'] >0), all_data['k_lost'], 0)
    
    odds = betting_return(result_dummies, all_data[bookmaker])
    all_data = pd.concat([all_data, odds],axis=1)
    
    all_data[['act_win','act_draw','act_lost']] =pd.DataFrame(odds.values*all_data[['bet_win','bet_draw','bet_lost']].values, columns=odds.columns, index=odds.index)   
    all_data['total'] = all_data['act_win']+all_data['act_draw']+all_data['act_lost']   

    actual_profit = (all_data['total']).sum()
    actual_betting_money = (all_data['bet_win']+all_data['bet_draw']+all_data['bet_lost']).sum()
    return all_data, actual_profit, actual_betting_money

In [ ]:
#5. K-Result/2: We bet one half of the amount bet by K-Result, bi = fiC/6k.
#6. K-Result/4: We bet one quarter of the amount bet by K-Result, bi = fiC/12k.

In [159]:
#method1 赢，平，负概率最大的下注
total_profit = 0
total_bet = 0
for file_name in filename_list:
    all_data, actual_profit, actual_betting_money = betting_with_max_p(file_name, bookmaker=['b365H', 'b365D', 'b365A'], bankroll_unit = 1)
    all_data.to_csv(rootpath+'method1/'+file_name,  encoding='utf-8', index=False)
    print('%s 实际盈利是: %f, 总下注金额是: %f' % (file_name, actual_profit, actual_betting_money))
    total_profit = total_profit+actual_profit
    total_bet = actual_betting_money+total_bet
print('十二轮总盈利：%f, 十二轮下注总金额：%f' %(total_profit, total_bet))

baidu-080815.csv 实际盈利是: -4.010000, 总下注金额是: 20.000000
baidu-081515.csv 实际盈利是: -4.200000, 总下注金额是: 29.000000
baidu-082215.csv 实际盈利是: -12.740000, 总下注金额是: 39.000000
baidu-082815.csv 实际盈利是: -1.860000, 总下注金额是: 49.000000
baidu-091215.csv 实际盈利是: 2.380000, 总下注金额是: 49.000000
baidu-091915.csv 实际盈利是: -0.080000, 总下注金额是: 49.000000
baidu-092315.csv 实际盈利是: -3.160000, 总下注金额是: 39.000000
baidu-092615.csv 实际盈利是: -2.600000, 总下注金额是: 48.000000
baidu-100315.csv 实际盈利是: -2.030000, 总下注金额是: 46.000000
baidu-101715.csv 实际盈利是: -9.090000, 总下注金额是: 48.000000
baidu-102415.csv 实际盈利是: 3.690000, 总下注金额是: 49.000000
baidu-120415.csv 实际盈利是: -14.230000, 总下注金额是: 47.000000
十二轮总盈利：-47.930000, 十二轮下注总金额：512.000000


In [160]:
#method2  概率 − 1/赔率 >0 均下注
total_profit = 0
total_bet = 0
for file_name in filename_list:
    all_data, actual_profit, actual_betting_money = betting_with_d_positive(file_name, bookmaker=['b365H', 'b365D', 'b365A'], bankroll_unit = 1)
    all_data.to_csv(rootpath+'method2/'+file_name,  encoding='utf-8', index=False)
    print('%s 实际盈利是: %f, 总下注金额是: %f' % (file_name, actual_profit, actual_betting_money))
    total_profit = total_profit+actual_profit
    total_bet = actual_betting_money+total_bet
print('十二轮总盈利：%f, 十二轮下注总金额：%f' %(total_profit, total_bet))    

baidu-080815.csv 实际盈利是: 9.300000, 总下注金额是: 18.000000
baidu-081515.csv 实际盈利是: -0.750000, 总下注金额是: 18.000000
baidu-082215.csv 实际盈利是: 12.350000, 总下注金额是: 21.000000
baidu-082815.csv 实际盈利是: -5.800000, 总下注金额是: 31.000000
baidu-091215.csv 实际盈利是: 2.050000, 总下注金额是: 28.000000
baidu-091915.csv 实际盈利是: -9.520000, 总下注金额是: 22.000000
baidu-092315.csv 实际盈利是: -13.900000, 总下注金额是: 17.000000
baidu-092615.csv 实际盈利是: 11.600000, 总下注金额是: 29.000000
baidu-100315.csv 实际盈利是: 0.650000, 总下注金额是: 18.000000
baidu-101715.csv 实际盈利是: -8.400000, 总下注金额是: 26.000000
baidu-102415.csv 实际盈利是: -8.450000, 总下注金额是: 23.000000
baidu-120415.csv 实际盈利是: 20.380000, 总下注金额是: 25.000000
十二轮总盈利：9.510000, 十二轮下注总金额：276.000000


In [161]:
#method3 每场比赛，D= 概率 − 1/赔率 >0的，选最大者下注。如Dhome_win > Dhome_lost >Dhome_draw时，且Dhome_win>0时，只下注Dhome_win
total_profit = 0
total_bet = 0
for file_name in filename_list:
    all_data, actual_profit, actual_betting_money = betting_with_max_d_positive(file_name, bookmaker=['b365H', 'b365D', 'b365A'], bankroll_unit = 1)
    all_data.to_csv(rootpath+'method3/'+file_name,  encoding='utf-8', index=False)
    print('%s 实际盈利是: %f, 总下注金额是: %f' % (file_name, actual_profit, actual_betting_money))
    total_profit = total_profit+actual_profit
    total_bet = actual_betting_money+total_bet
print('十二轮总盈利：%f, 十二轮下注总金额：%f' %(total_profit, total_bet))

baidu-080815.csv 实际盈利是: 2.300000, 总下注金额是: 13.000000
baidu-081515.csv 实际盈利是: 1.250000, 总下注金额是: 16.000000
baidu-082215.csv 实际盈利是: 9.100000, 总下注金额是: 19.000000
baidu-082815.csv 实际盈利是: -11.300000, 总下注金额是: 22.000000
baidu-091215.csv 实际盈利是: 1.050000, 总下注金额是: 25.000000
baidu-091915.csv 实际盈利是: -8.520000, 总下注金额是: 21.000000
baidu-092315.csv 实际盈利是: -10.900000, 总下注金额是: 14.000000
baidu-092615.csv 实际盈利是: 16.600000, 总下注金额是: 24.000000
baidu-100315.csv 实际盈利是: 0.150000, 总下注金额是: 15.000000
baidu-101715.csv 实际盈利是: -6.400000, 总下注金额是: 24.000000
baidu-102415.csv 实际盈利是: -14.250000, 总下注金额是: 21.000000
baidu-120415.csv 实际盈利是: 18.630000, 总下注金额是: 23.000000
十二轮总盈利：-2.290000, 十二轮下注总金额：237.000000


In [162]:
#method4 凯利指数大于0，均下注
total_profit = 0
total_bet = 0
for file_name in filename_list:
    all_data, actual_profit, actual_betting_money = betting_with_k_result(file_name, bookmaker=['b365H', 'b365D', 'b365A'], bankroll = 1000)
    all_data.to_csv(rootpath+'method4/'+file_name,  encoding='utf-8', index=False)
    print('%s 实际盈利是: %f, 总下注金额是: %f' % (file_name, actual_profit, actual_betting_money))
    total_profit = total_profit+actual_profit
    total_bet = actual_betting_money+total_bet
print('十二轮总盈利：%f, 十二轮下注总金额：%f' %(total_profit, total_bet))    

baidu-080815.csv 实际盈利是: 1.853030, 总下注金额是: 2.946986
baidu-081515.csv 实际盈利是: -0.620722, 总下注金额是: 1.985566
baidu-082215.csv 实际盈利是: 1.567384, 总下注金额是: 2.002967
baidu-082815.csv 实际盈利是: -0.957408, 总下注金额是: 2.142128
baidu-091215.csv 实际盈利是: -0.471145, 总下注金额是: 2.312639
baidu-091915.csv 实际盈利是: -0.216645, 总下注金额是: 1.553187
baidu-092315.csv 实际盈利是: -0.991694, 总下注金额是: 1.281886
baidu-092615.csv 实际盈利是: 2.380704, 总下注金额是: 2.182290
baidu-100315.csv 实际盈利是: 0.028059, 总下注金额是: 1.246352
baidu-101715.csv 实际盈利是: -0.540081, 总下注金额是: 1.907627
baidu-102415.csv 实际盈利是: -1.687577, 总下注金额是: 2.050912
baidu-120415.csv 实际盈利是: 1.230228, 总下注金额是: 1.969907
十二轮总盈利：1.574134, 十二轮下注总金额：23.582446
